# Guider mode image reduction
This is intended to unpack the rawStamps from guider mode FITS files and build a movie of all 16 stamps.\
Craig Lage - 03-Dec-24

In [ ]:
import os
import shlex, subprocess
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from astropy.io import fits
from lsst.resources import ResourcePath

In [ ]:
# Example of getting a FITS file stored at USDF
# LSSTComCam science CCD
filename = "s3://rubin-summit/LSSTComCam/20240807/CC_O_20240807_000009/CC_O_20240807_000009_R22_S00_guider.fits"

# LSSTcam guider CCD
#filename = "s3://embargo@rubin-summit/LSSTCam/20241008/MC_C_20241008_000025/MC_C_20241008_000025_R00_SG0_guider.fits"

rp = ResourcePath(filename)
with rp.open(mode="rb") as f:
    hdu_list = fits.open(f)
len(hdu_list)

# Get the main header and the information it contains

In [ ]:
def getMainHeaderInfo(hdu_list):
    hdr0 = hdu_list[0].header
    #raft = hdr0['RAFTBAY']
    #ccd = hdr0['CCDSLOT']
    #dayObs = hdr0['DAYOBS']
    #seqNum = hdr0['SEQNUM']
    roiCols = hdr0['ROICOLS']
    roiRows = hdr0['ROIROWS']
    try:
        roiUnder = hdr0['ROIUNDER']
    except:
        roiUnder = 6
    nStamps = hdr0['N_STAMPS']
    
    # Set the xor value - Guider CCDs are different from science CCDs
    if raft in ['R00', 'R04', 'R40', 'R44']:
        # Guider rafts
        xor = 0x20000
    else:
        # Science rafts
        xor = 0x1ffff
    return [roiRows, roiCols, roiUnder, nStamps, xor]

# Now define the code to unpack the rawStamps:

In [ ]:
def unpackStamps(hduNum):
    data = (hdu_list[hduNum].data['rawStamp'][0]).astype('>u4')
    data.byteswap(inplace=True)
    totalCols = roiCols + roiUnder
    size = roiRows * totalCols
    out = np.zeros([16, size], dtype=int)
    image_out = np.zeros([16, roiRows, roiCols], dtype=int)
    
    for n in range(size):
        # Get 9 32 bit words of data
        res = ''
        for i in range(9):
            d = data[(size - n) * 9 - i - 1]
            d = format(d, '#034b')
            d = d.split('b')[1]
            res += d
        # Now extract 16 18 bit words from the data
        for i in range(16):
            bin_value = res[i * 18:(i + 1) * 18]
            int_value = int(bin_value, 2)
            final_value = int_value ^ xor
            out[i,n] = final_value  
    for i in range(16):
        reshaped = out[i,:].reshape(roiRows, totalCols)
        image_out[i,:,:] = np.flipud(np.fliplr(reshaped[:,0:roiCols]))
    return image_out

In [ ]:
# Set the scaling
autoscale = True
# Scale to use if autoscale = False
vmin = 10000
vmax = 30000

# Now build the individual movie frames

In [ ]:
dayObs = 20241204
seqNum = 134
expId = int(f"{dayObs}{seqNum:05d}")
raft = 'R22'
ccd = 'S22'

#filename = f"s3://rubin-summit/LSSTComCam/{dayObs}/CC_O_{dayObs}_{seqNum:06d}/CC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"
filename = f"s3://embargo@rubin-summit/LSSTComCam/{dayObs}/CC_O_{dayObs}_{seqNum:06d}/CC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits" 
rp = ResourcePath(filename)
with rp.open(mode="rb") as f:
    hdu_list = fits.open(f)
[roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)

dirName = f"/home/c/cslage/u/Guider_Mode/movie_{expId}_{raft}_{ccd}"
%mkdir -p {dirName}
movieName = f"Guider_{expId}_{raft}_{ccd}.mp4"
print(movieName)
[roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
# Build the individual frames
fig = plt.figure(figsize=(10,10))
for n in range(nStamps):
    hduNum = 2 * n + 1
    hdrn = hdu_list[hduNum].header
    timestamp = hdrn['STMPTIME']
    image_out = unpackStamps(hduNum)
    axs = fig.subplots(4,4)
    plt.subplots_adjust(wspace=.7, hspace=-0.2)
    plt.suptitle(f"Guider mode {raft} {ccd} {expId}, Frame {n+1}\n{timestamp}", fontsize=24)
    for i in range(4):
        for j in range(4):
            seg = i + 4 * j
            axs[i][j].set_title(f"segment {seg}")
            if autoscale:
                im = axs[i][j].imshow(image_out[seg], interpolation='nearest', origin='upper', cmap='Greys')
            else:
                im = axs[i][j].imshow(image_out[seg], interpolation='nearest', origin='upper', vmin=vmin, vmax=vmax, cmap='Greys')
            divider = make_axes_locatable(axs[i][j])
            cax = divider.append_axes("right", size="5%", pad=0.05)
            fig.colorbar(im, cax=cax)
    plt.savefig(f"{dirName}/Frame_{n:03d}.png")
    plt.clf()
    if n % 10 == 0:
        print(f"Finished frame {n}")
print("Done building frames")

# Now make the movie

In [ ]:
print(f"\033[1mThe movie name will be: {dirName}/{movieName}\033[0m")

command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -vcodec libx264 -pix_fmt yuv420p -framerate 50 -y {dirName}/{movieName}"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()

# Comparing Unpacked BINTABLE to the corresponding IMAGE header

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(12,5))
plt.subplots_adjust(hspace=1.0)
plt.suptitle(f"Guider mode comparison {raft} {expId}", fontsize=18)
data = hdu_list[6].data
axs[0].set_title("IMAGE header 6")
im1 = axs[0].imshow(data, interpolation='nearest', vmin=10000, vmax=30000, origin='upper')
axs[0].text(38,4, '+', color='black')
divider = make_axes_locatable(axs[0])
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im1, cax=cax)

image_out = unpackStamps(5)
axs[1].set_title("Unpacked BINTABLE header 5 - Seg13")
im2 = axs[1].imshow(image_out[13], interpolation='nearest', vmin=10000, vmax=30000, origin='upper')
axs[1].text(38,4, '+', color='black')
divider = make_axes_locatable(axs[1])
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im2, cax=cax)
plt.savefig(f"/home/c/cslage/u/Guider_Mode/Unpack_Comparison_{expId}_{raft}_{ccd}.png")

# The two images are bitwise identical
diff = data - image_out[13]
print(f"Diff max = {np.max(diff)}, Diff min = {np.min(diff)}")

In [ ]:
axs

# Movie of a single frame

In [ ]:
dayObs = 20241204
seqNum = 134
expId = int(f"{dayObs}{seqNum:05d}")
raft = 'R22'
ccd = 'S22'
seg = 13 

filename = f"s3://embargo@rubin-summit/LSSTComCam/{dayObs}/CC_O_{dayObs}_{seqNum:06d}/CC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits" 
rp = ResourcePath(filename)
with rp.open(mode="rb") as f:
    hdu_list = fits.open(f)
[roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)

dirName = f"/home/c/cslage/u/Guider_Mode/single_movie_{expId}_{raft}_{ccd}_{seg}"
%mkdir -p {dirName}
movieName = f"Guider_{expId}_{raft}_{ccd}_{seg}.mp4"
print(movieName)
frames = []
meds = []
[roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
# Build the individual frames
fig = plt.figure(figsize=(10,10))
for n in range(nStamps):
    hduNum = 2 * n + 1
    hdrn = hdu_list[hduNum].header
    timestamp = hdrn['STMPTIME']
    image_out = unpackStamps(hduNum)
    ax = fig.subplots(1,1)
    ax.set_title(f"Guider mode {raft} {ccd} {expId} {seg}, Frame {n+1}\n{timestamp}", fontsize=24)
    arr = np.array(image_out[seg])
    arr = arr[0:125,0:125]
    meds.append(np.median(arr))
    arr = arr - np.median(arr)
   
    
    #print(np.max(arr))
    #break
    (ycen, xcen) = np.unravel_index(arr.argmax(), arr.shape)
    stamp = 30
    x1 = int(xcen - stamp / 2)
    x2 = x1 + stamp
    
    y1 = int(ycen - stamp / 2)
    y2 = y1 + stamp
    im = arr[y1:y2, x1:x2]
    
    #im = arr
    frames.append(im)
    #print(x1, x2, y1, y2, xcen, ycen)
    ax.imshow(im, interpolation='nearest', origin='lower', vmin=0, vmax=10000)#, cmap='Greys')
    #divider = make_axes_locatable(ax)
    #cax = divider.append_axes("right", size="5%", pad=0.05)
    #fig.colorbar(im, cax=cax)
    plt.savefig(f"{dirName}/Frame_{n:03d}.png")
    plt.clf()
    if n % 10 == 0:
        print(f"Finished frame {n}")
print("Done building frames")

# Now make the movie

In [ ]:
print(f"\033[1mThe movie name will be: {dirName}/{movieName}\033[0m")

command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -vcodec libx264 -pix_fmt yuv420p -framerate 50 -y {dirName}/{movieName}"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,5))
plt.suptitle(f"Guider mode {raft} {ccd} {expId} {seg}", fontsize=18)
fluxes = []
for frame in frames:
    axs[0].plot(frame[15,0:30])
    fluxes.append(np.sum(frame))
axs[0].set_title("Flux scans by frame")
axs[1].set_title("Histogram of fluxes")

axs[1].hist(fluxes, bins=50)
plt.savefig(f"/home/c/cslage/u/Guider_Mode/analysis/Guider_Fluxes_{expId}_{raft}_{ccd}.png")

In [ ]:
hduNum = 1
hdr_test = hdu_list[hduNum].header
for key in hdr_test.keys():
    print(key, hdr_test[key])

In [ ]:
plt.hist(meds)

In [ ]:
n = 2
hduNum = 4 * n + 2
arr = hdu_list[hduNum].data
ax = fig.subplots(1,1)
ax.set_title(f"Guider mode raw {raft} {ccd} {expId} {seg}, Frame {n+1}\n{timestamp}", fontsize=24)
arr = arr[50:200,50:200]
meds.append(np.median(arr))
arr = arr - np.median(arr)
plt.imshow(arr, interpolation='nearest', origin='lower')


# Movie of a single frame - rawStamps

In [ ]:
dayObs = 20241204
seqNum = 129
expId = int(f"{dayObs}{seqNum:05d}")
raft = 'R22'
ccd = 'S22'
seg = 13 

filename = f"s3://embargo@rubin-summit/LSSTComCam/{dayObs}/CC_O_{dayObs}_{seqNum:06d}/CC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits" 
rp = ResourcePath(filename)
with rp.open(mode="rb") as f:
    hdu_list = fits.open(f)
[roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)

dirName = f"/home/c/cslage/u/Guider_Mode/single_movie_raw_{expId}_{raft}_{ccd}_{seg}"
%mkdir -p {dirName}
movieName = f"Guider_raw_{expId}_{raft}_{ccd}_{seg}.mp4"
print(movieName)
frames = []
meds = []
[roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
# Build the individual frames
fig = plt.figure(figsize=(10,10))
for n in range(int(nStamps/2)):
    hduNum = 4 * n + 2
    arr = hdu_list[hduNum].data
    ax = fig.subplots(1,1)
    ax.set_title(f"Guider mode raw {raft} {ccd} {expId} {seg}, Frame {n+1}\n{timestamp}", fontsize=24)
    arr = arr[50:200,50:200]
    meds.append(np.median(arr))
    arr = arr - np.median(arr)
    (ycen, xcen) = np.unravel_index(arr.argmax(), arr.shape)
    stamp = 30
    x1 = int(xcen - stamp / 2)
    x2 = x1 + stamp
    
    y1 = int(ycen - stamp / 2)
    y2 = y1 + stamp
    im = arr[y1:y2, x1:x2]
    
    #im = arr
    frames.append(im)
    #print(x1, x2, y1, y2, xcen, ycen)
    ax.imshow(im, interpolation='nearest', origin='lower', vmin=0, vmax=30000)#, cmap='Greys')
    #divider = make_axes_locatable(ax)
    #cax = divider.append_axes("right", size="5%", pad=0.05)
    #fig.colorbar(im, cax=cax)
    plt.savefig(f"{dirName}/Frame_{n:03d}.png")
    plt.clf()
    if n % 10 == 0:
        print(f"Finished frame {n}")
print("Done building frames")

# Now make the movie

In [ ]:
print(f"\033[1mThe movie name will be: {dirName}/{movieName}\033[0m")

command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -vcodec libx264 -pix_fmt yuv420p -framerate 50 -y {dirName}/{movieName}"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,5))
plt.suptitle(f"Guider mode {raft} {ccd} {expId} {seg}", fontsize=18)
fluxes = []
for frame in frames:
    axs[0].plot(frame[15,0:30])
    fluxes.append(np.sum(frame))
axs[0].set_title("Flux scans by frame")
axs[1].set_title("Histogram of fluxes")

axs[1].hist(fluxes, bins=50)
plt.savefig(f"/home/c/cslage/u/Guider_Mode/analysis/Guider_Fluxes_Raw_{expId}_{raft}_{ccd}.png")